## 순환신경망 구현 및 학습

In [1]:
import tensorflow as tf

## 하이퍼 파라미터

In [2]:
EPOCHS = 10
NUM_WORDS = 10000

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.emb = tf.keras.layers.Embedding(NUM_WORDS, 32)
    self.rnn = tf.keras.layers.LSTM(100)
    self.dense = tf.keras.layers.Dense(2, activation='softmax')

  def call(self, x, training=None, mask=None):
    x = self.emb(x)
    x = self.rnn(x)
    return self.dense(x)

## 학습, 테스트 루프 정의

In [4]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs, labels, loss_object, test_loss, test_accuracy):
    predictions = model(inputs, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [5]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words= NUM_WORDS)

print("data 별로 길이가 다름")
print(len(x_train[0]))
print(len(x_train[1]))
print(len(x_train[2]))

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                       value = 0,
                                                       padding='pre',
                                                       maxlen=200)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                       value = 0,
                                                       padding='pre',
                                                       maxlen=200)
# Y label일 경우 padding은 post로 해주면 됨. 하지만, 여기선 label이 0 또는 1이므로 해줄 필요 없음

print("padding을 통해 data별 길이 맞춰줌")
print(len(x_train[0]))
print(len(x_train[1]))
print(len(x_train[2]))

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

data 별로 길이가 다름
218
189
141
padding을 통해 data별 길이 맞춰줌
200
200
200


## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [6]:
# Create model
model = MyModel()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [7]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.44218701124191284, Accuracy: 78.96400451660156, Test Loss: 0.3341729938983917, Test Accuracy: 85.78399658203125
Epoch 2, Loss: 0.2619674801826477, Accuracy: 89.56400299072266, Test Loss: 0.3137684166431427, Test Accuracy: 86.87200164794922
Epoch 3, Loss: 0.19660374522209167, Accuracy: 92.5320053100586, Test Loss: 0.3385576903820038, Test Accuracy: 86.29199981689453
Epoch 4, Loss: 0.1544586569070816, Accuracy: 94.20800018310547, Test Loss: 0.4021235406398773, Test Accuracy: 83.97200012207031
Epoch 5, Loss: 0.1217561885714531, Accuracy: 95.54000091552734, Test Loss: 0.4395184814929962, Test Accuracy: 85.38800048828125
Epoch 6, Loss: 0.0888611301779747, Accuracy: 96.94000244140625, Test Loss: 0.5182793736457825, Test Accuracy: 84.9280014038086
Epoch 7, Loss: 0.07079796493053436, Accuracy: 97.552001953125, Test Loss: 0.5122295618057251, Test Accuracy: 84.80799865722656
Epoch 8, Loss: 0.05156628414988518, Accuracy: 98.3239974975586, Test Loss: 0.5979423522949219, Test Accur